#Introduction

Business intelligence is very important nowadays. It helps you to know better of your company including how the sales recently, which item is going to out of stock and most importantly, what is the profit.

In this project, I am going to demonstrate how to use data from a pizza shop along with SQL to create 3 dashboards for business intelligence.

##Data content

There are 10 tables in the dataset.

Table name | Content | Columns
--- | --- | ---
Address | Customers' address and zip code | 'add_id', 'delivery_address1', 'delivery_address2', 'delivery_city', 'delivery_zipcode'
Customers | First and last name | 'cust_id', 'cust_firstname', 'cust_lastname'
Ingredient | Ingredient name, weight, unit and price | 'ing_id', 'ing_name', 'ing_weight', 'ing_meas', 'ing_price'
Inventory | Quantity of inventory | 'inv_id', 'item_id', 'quantity'
Item | Item name, code, category, size and price | 'item_id', 'sku', 'item_name', 'item_cat', 'item_size', 'item_price'
Orders | Order create time, item ordered, quantity, customer ID, delivery/pick up, address ID | 'row_id', 'order_id', 'created_at', 'item_id', 'quantity', 'cust_id', 'delivery', 'add_id'
Recipe | Recipe ID, ingredient ID, required quantity | 'row_id', 'recipe_id', 'ing_id', 'quantity'
Rota | Duty date, shift ID, staff ID | 'row_id', 'rota_id', 'date', 'shift_id', 'staff_id'
Shift | Day of week, start time and end time | 'shift_id', 'day_of_week', 'start_time', 'end_time'
Staff | Staff first and last name, position, hourly rate | 'staff_id', 'first_name', 'last_name', 'position', 'hourly_rate'

##Dashboard 1

This dashboard contains the basic information about the pizza shop. It tells whether the pizza shop is earning or losing money.
1.   Total number of order
2.   Total sales
3.   Total item sold
4.   Average order value
5.   Sales by category
6.   Top selling item
7.   Orders by hour
8.   Sales by hour
9.   Orders by address
10.  Orders by delivery/pick up

##Dashboard 2

The second dashboard is about the material cost and stock information to keep the pizza shop running.
1.   Total quantity required for orders by ingredient
2.   Total cost of ingredients for orders
3.   Calculated cost of pizza
4.   Percentage stock remaining by ingredient
5.   List of ingredients to re-order based on remaining inventory

##Dashboard 3

The last dashboard is going to capture the human resource cost of the pizza shop.
1.   Hours worked by staff member
2.   Total hours worked
3.   Cost per staff member
4.   Total staff cost

##Import data

In [ ]:
# connect google drive
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# import library
import sqlite3
import pandas as pd
import numpy as np

In [ ]:
# define functions

def pd_to_sqlDB(input_df: pd.DataFrame, table_name: str, db_name: str = 'default.db') -> None:

    '''Take a Pandas dataframe `input_df` and upload it to `table_name` SQLITE table
    Args:
        input_df (pd.DataFrame): Dataframe containing data to upload to SQLITE
        table_name (str): Name of the SQLITE table to upload to
        db_name (str, optional): Name of the SQLITE Database in which the table is created. 
                                 Defaults to 'default.db'.
    '''

    # Step 1: Find columns in the dataframe
    cols = input_df.columns
    cols_string = ','.join(cols)
    val_wildcard_string = ','.join(['?'] * len(cols))

    # Step 2: Connect to a DB file if it exists, else crete a new file
    con = sqlite3.connect(db_name)
    cur = con.cursor()

    # Step 3: Create Table
    sql_string = f"""CREATE TABLE IF NOT EXISTS {table_name} ({cols_string});"""
    cur.execute(sql_string)

    # Step 4: Upload the dataframe
    rows_to_upload = input_df.to_dict(orient='split')['data']
    sql_string = f"""INSERT INTO {table_name} ({cols_string}) VALUES ({val_wildcard_string});"""    
    cur.executemany(sql_string, rows_to_upload)
  
    # Step 5: Commit the changes and close the connection
    con.commit()
    con.close()



def sql_query_to_pd(sql_query_string: str, db_name: str ='default.db') -> pd.DataFrame:
    '''Execute an SQL query and return the results as a pandas dataframe
    Args:
        sql_query_string (str): SQL query string to execute
        db_name (str, optional): Name of the SQLITE Database to execute the query in.
                                 Defaults to 'default.db'.
    Returns:
        pd.DataFrame: Results of the SQL query in a pandas dataframe
    '''    
    # Step 1: Connect to the SQL DB
    con = sqlite3.connect(db_name)

    # Step 2: Execute the SQL query
    cursor = con.execute(sql_query_string)

    # Step 3: Fetch the data and column names
    result_data = cursor.fetchall()
    cols = [description[0] for description in cursor.description]

    # Step 4: Close the connection
    con.close()

    # Step 5: Return as a dataframe
    return pd.DataFrame(result_data, columns=cols)

In [ ]:
# import data
address=pd.read_csv('/content/gdrive/My Drive/dataset/SQL/pizza_proj/address.csv')
customers=pd.read_csv('/content/gdrive/My Drive/dataset/SQL/pizza_proj/customers.csv')
ingredient=pd.read_csv('/content/gdrive/My Drive/dataset/SQL/pizza_proj/ingredient.csv')
inventory=pd.read_csv('/content/gdrive/My Drive/dataset/SQL/pizza_proj/inventory.csv')
item=pd.read_csv('/content/gdrive/My Drive/dataset/SQL/pizza_proj/item.csv')
orders=pd.read_csv('/content/gdrive/My Drive/dataset/SQL/pizza_proj/orders.csv')
recipe=pd.read_csv('/content/gdrive/My Drive/dataset/SQL/pizza_proj/recipe.csv')
rota=pd.read_csv('/content/gdrive/My Drive/dataset/SQL/pizza_proj/rota.csv')
shift=pd.read_csv('/content/gdrive/My Drive/dataset/SQL/pizza_proj/shift.csv')
staff=pd.read_csv('/content/gdrive/My Drive/dataset/SQL/pizza_proj/staff.csv')

In [ ]:
# import data to sql tables
pd_to_sqlDB(address,"address","pizza_db")
pd_to_sqlDB(customers,"customers","pizza_db")
pd_to_sqlDB(ingredient,"ingredient","pizza_db")
pd_to_sqlDB(inventory,"inventory","pizza_db")
pd_to_sqlDB(item,"item","pizza_db")
pd_to_sqlDB(orders,"orders","pizza_db")
pd_to_sqlDB(recipe,"recipe","pizza_db")
pd_to_sqlDB(rota,"rota","pizza_db")
pd_to_sqlDB(shift,"shift","pizza_db")
pd_to_sqlDB(staff,"staff","pizza_db")

#Query for 1st dashboard

In [ ]:
# Query table for the 1st dashboard
query="""
SELECT
  o.order_id,
  o.quantity*i.item_price AS order_sales,
  o.quantity,
  i.item_cat,
  i.item_name,
  o.created_at,
  a.delivery_address1,
  -- a.delivery_address2, --no data in address 2
  a.delivery_city,
  a.delivery_zipcode,
  o.delivery
FROM orders o
LEFT JOIN item i ON o.item_id=i.item_id
LEFT JOIN address a ON o.add_id=a.add_id;
"""
result1=sql_query_to_pd(query,"pizza_db")

In [ ]:
# Convert to appropriate data type
result1['created_at']=pd.to_datetime(result1['created_at'],format="%d/%m/%Y %H:%M")
result1['delivery_zipcode']=result1['delivery_zipcode'].astype('str')

In [ ]:
# Table for 1st dashboard 
print(result1)

##Dashboard 1

This dashboard contains the basic information about the pizza shop. It tells whether the pizza shop is earning or losing money.
1.   Total number of order
2.   Total sales
3.   Total item sold
4.   Average order value
5.   Sales by category
6.   Top selling item
7.   Orders by hour
8.   Sales by hour
9.   Orders by address
10.  Orders by delivery/pick up

##1. Total orders

In [ ]:
ttl_no_order=len(result1['order_id'].unique())
print(f'The total number of order is {ttl_no_order}')

##2. Total sales

In [ ]:
ttl_sales=sum(result1['order_sales'])
print(f'The total sales is ${ttl_sales}')

##3. Total item sold

In [ ]:
ttl_item_sold=sum(result1['quantity'])
print(f'The total item sold is {ttl_item_sold}')

##4. Average order value

In [ ]:
avg_order_val=result1.groupby(['order_id'])['order_sales'].sum().mean()
print(f'The average order value is ${avg_order_val:.2f}')

##5. Sales by category

In [ ]:
sales_by_cat=result1.groupby(['item_cat'])['order_sales'].sum().to_frame().sort_values('order_sales',ascending=False)
print(f'The average order value is \n',sales_by_cat)

##6. Top selling item

In [ ]:
top_item=result1.groupby(['item_name'])['order_sales'].sum().idxmax()
print(f'The top selling item is {top_item}')

##7. Order by hour

In [ ]:
order_by_hour=result1.groupby(result1['created_at'].dt.hour)['order_id'].count().to_frame('count').rename_axis('hour')
print(f'The order by hour is\n',order_by_hour)

##8. Sales by hour

In [ ]:
sales_by_hour=result1.groupby(result1['created_at'].dt.hour)['order_sales'].sum().to_frame('sales').rename_axis('hour')
print(f'The sales by hour is\n',sales_by_hour)

##9. Order by address

In [ ]:
order_by_address=result1['delivery_address1'].value_counts().to_frame('count').rename_axis('address')
print(f'The order by address is\n',order_by_address)

##10. Order by delivery/pick up

In [ ]:
order_by_delivery=result1['delivery'].value_counts().to_frame('count').rename_axis('delivery(1)/pick up(0)')
print(f'The order by delivery/pick up is\n',order_by_delivery)

#Query for 2nd dashboard

In [ ]:
# Query table for the 2nd dashboard
query="""
SELECT
  sub1.item_name,
  sub1.ing_id,
  sub1.ing_name,
  sub1.ing_weight,
  sub1.ing_price,
  sub1.order_quan,
  sub1.rep_quan,
  sub1.order_quan*sub1.rep_quan AS quantity_required,
  sub1.ing_price/sub1.ing_weight AS unit_cost,
  (sub1.order_quan*sub1.rep_quan)*(sub1.ing_price/sub1.ing_weight) AS ingredient_cost
FROM
    (SELECT
      o.item_id,
      rep.recipe_id,
      i.item_name,
      rep.ing_id,
      ing.ing_name,
      SUM(o.quantity)  AS order_quan,
      rep.quantity AS rep_quan,
      ing.ing_price,
      ing.ing_weight
    FROM orders o
    LEFT JOIN item i ON o.item_id=i.item_id
    LEFT JOIN recipe rep ON i.sku=rep.recipe_id
    LEFT JOIN ingredient ing ON rep.ing_id=ing.ing_id
    GROUP BY
      o.item_id, 
      rep.recipe_id, 
      i.item_name,
      rep.ing_id,
      rep.quantity,
      ing.ing_name,
      ing.ing_weight,
      ing.ing_price) sub1;
"""
result2=sql_query_to_pd(query,"pizza_db")
result2['item_cost']=result2['rep_quan']*result2['unit_cost']
print(result2)

##Dashboard 2

The second dashboard is about the material cost and stock information to keep the pizza shop running.
1.   Total quantity required for orders by ingredient
2.   Total cost of ingredients for orders
3.   Calculated cost of pizza
4.   Percentage stock remaining by ingredient
5.   List of ingredients to re-order based on remaining inventory

##1. Total quantity required for orders by ingredient

In [ ]:
quantity_by_ingredient=result2.groupby('ing_name')['quantity_required'].sum().to_frame()
print(f'The total quantity required for orders by ingredient is\n',quantity_by_ingredient)

##2. Total cost of ingredients for orders

In [ ]:
cost_by_ingredient=result2.groupby('ing_name')['ingredient_cost'].sum().to_frame()
print(f'The total cost of ingredients for orders is\n',cost_by_ingredient.round(2))

##3. Calculated cost of pizza

In [ ]:
cost_of_pizza=result2.groupby('item_name')[['item_cost']].sum()
filt=(cost_of_pizza.index.str.contains("Pizza"))
print(f'The cost_of_pizza is\n',cost_of_pizza[filt].round(2))

In [ ]:
# use previous query for following questions
pd_to_sqlDB(result2,"result2","pizza_db")

# query to answer remaining question in dashboard2
query="""
SELECT
  sub2.ing_name,
  sub2.ordered_weight,
  (ing.ing_weight*inv.quantity) AS total_inv_weight
FROM
  (SELECT
    ing_id,
    ing_name,
    SUM(quantity_required) AS ordered_weight
  FROM result2
  GROUP BY 
  ing_name, ing_id) sub2
LEFT JOIN inventory inv ON inv.item_id = sub2.ing_id
LEFT JOIN ingredient ing ON ing.ing_id = sub2.ing_id;
"""
result3=sql_query_to_pd(query,"pizza_db")
print(result3)

##4. Percentage stock remaining by ingredient

In [ ]:
result3['perc_remain']=((result3['total_inv_weight']-result3['ordered_weight'])/result3['total_inv_weight']*100)
print(f'The percentage remaining is\n',result3[['ing_name','perc_remain']].round(2))

##5. List of ingredients to re-order based on remaining inventory

In [ ]:
print(f'The ingredients need to re-order is\n',result3[['ing_name','perc_remain']].sort_values("perc_remain").round(2))

#Query for 3rd dashboard

In [ ]:
# Query table for the 3rd dashboard
query="""
SELECT
  st.first_name,
  st.last_name,
  st.hourly_rate,
  sh.day_of_week,
  sh.start_time,
  sh.end_time
FROM staff st
LEFT JOIN rota ro ON ro.staff_id=st.staff_id
LEFT JOIN shift sh ON sh.shift_id=ro.shift_id
"""
result4=sql_query_to_pd(query,"pizza_db")
result4['start_time']=pd.to_timedelta(result4['start_time'].str.strip())
result4['end_time']=pd.to_timedelta(result4['end_time'].str.strip())
result4['hour_worked']=(result4['end_time']-result4['start_time'])/np.timedelta64(1, 'h')
print(result4)

##Dashboard 3

The last dashboard is going to capture the human resource cost of the pizza shop.
1.   Hours worked by staff member
2.   Total hours worked
3.   Cost per staff member
4.   Total staff cost

In [ ]:
hours_worked_by_staff=result4.groupby(['first_name'])['hour_worked'].sum()
print(f'The hours worked by staff is\n',hours_worked_by_staff)

##2. Total hours worked

In [ ]:
ttl_hour_worked=hours_worked_by_staff.sum()
print(f'The total hour worked is {ttl_hour_worked}')

##3. Cost per staff member

In [ ]:
cost_tab=result4.groupby(['first_name'])[['hour_worked','hourly_rate']].agg({'hour_worked':'sum','hourly_rate':'mean'})
cost_tab['cost']=cost_tab['hour_worked'].multiply(cost_tab['hourly_rate']).round(2)
print(f'The cost per staff is \n',cost_tab)

##4. Total staff cost

In [ ]:
ttl_staff_cost=cost_tab['cost'].sum()
print(f'The total staff cost is ${ttl_staff_cost}')

#Export csv for visualization

In [ ]:
result1.to_csv('/content/gdrive/My Drive/SQL/export/pizza1.csv',index=False)
result2.to_csv('/content/gdrive/My Drive/SQL/export/pizza2.csv',index=False)
result3.to_csv('/content/gdrive/My Drive/SQL/export/pizza3.csv',index=False)
result4.to_csv('/content/gdrive/My Drive/SQL/export/pizza4.csv',index=False)

link for visualization<br>
https://public.tableau.com/views/Book1_16823748427290/PizzaShopBI?:language=en-US&publish=yes&:display_count=n&:origin=viz_share_link

#Data source

https://learnbi.online/pizzaproject